In [ ]:
# https://medium.com/analytics-vidhya/exploring-langchain-agents-with-memory-basic-concepts-and-hands-on-code-eff6ea6fd411

In [ ]:
!pip install langchain --quiet
!pip install langchain_openai --quiet
!pip install langchain_chroma --quiet
!pip install wikipedia --quiet
!pip install duckduckgo-search --quiet
!pip install openai --quiet
!pip install tiktoken --quiet
!pip install google-search-results --quiet
!pip install python-dotenv --quiet
!pip install langchain_community --quiet

ERROR: Could not find a version that satisfies the requirement langchain_dash (from versions: none)
ERROR: No matching distribution found for langchain_dash


In [ ]:
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool
wikipedia = WikipediaAPIWrapper()



##Defining the tool

wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

wikipedia.run('History of Psychology')

'Page: History of psychology\nSummary: Psychology is defined as "the scientific study of behavior and mental processes". Philosophical interest in the human mind and behavior dates back to the ancient civilizations of Egypt, Persia, Greece, China, and India.\nPsychology as a field of experimental study began in 1854 in Leipzig, Germany, when Gustav Fechner created the first theory of how judgments about sensory experiences are made and how to experiment on them. Fechner\'s theory, recognized today as Signal Detection Theory, foreshadowed the development of statistical theories of comparative judgment and thousands of experiments based on his ideas (Link, S. W. Psychological Science, 1995). In 1879, Wilhelm Wundt founded the first psychological laboratory dedicated exclusively to psychological research in Leipzig, Germany. Wundt was also the first person to refer to himself as a psychologist. A notable precursor to Wundt was Ferdinand Ueberwasser (1752–1812), who designated himself Prof

In [ ]:
from langchain.agents import tool

@tool
def add_numbers(numbers):
    """Add two numbers"""
    numbers = numbers.replace(',', ' ').split()

    # Add input validation
    if len(numbers) == 2 and all(n.replace('.', '', 1).isdigit() for n in numbers):
        num1, num2 = map(float, numbers)
        return num1 + num2
    else:
        return "Please input two numbers separated by a comma."


add_numbers.run("10,40")

50.0

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()



duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

search.run("Future of Generative AI?")

'Making the world a better place is one thing. But perhaps a humbler prediction is that generative AI will make work better. Yes, some jobs will be lost to AI automation. But more jobs will be ... Speaking at the "Generative AI: Shaping the Future" symposium on Nov. 28, the kickoff event of MIT\'s Generative AI Week, keynote speaker and iRobot co-founder Rodney Brooks warned attendees against uncritically overestimating the capabilities of this emerging technology, which underpins increasingly powerful tools like OpenAI\'s ChatGPT and Google\'s Bard. By 2028, 30% of GenAI implementations will be optimized using energy-conserving computational methods, driven by sustainability initiatives. The rapid adoption of generative AI tools has made the negative environmental impact of GenAI, which the public and governments are calling out, an immediate concern for business leaders. One promising future direction Isola sees for generative AI is its use for fabrication. Instead of having a model 

In [ ]:
tools = []

tools.append(add_numbers)
tools.append(duckduckgo_tool)
tools.append(wikipedia_tool)

In [ ]:
import os
from langchain_openai import ChatOpenAI

from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')


os.environ["OPENAI_API_KEY"] = api_key
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")



In [ ]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {"input": "Find the heights of the world top 2 tallest building. Add the heights"},handle_parsing_errors=True
)

I need to find the heights of the world's top 2 tallest buildings and then add them together.
Action: DuckDuckGo Search
Action Input: "heights of world's top 2 tallest buildings"A comprehensive list of the 86 buildings in the world that are at least 350 metres (1,150 ft) tall, as of June 2024. Learn about the history, criteria, and rankings of skyscrapers, such as the Burj Khalifa, Merdeka 118, and Shanghai Tower. A list of the tallest skyscrapers in the world by standard height, with other details like floor count, observation decks, and year built. See the current rankings, locations, and features of the megatall and supertall buildings, such as Burj Khalifa, Merdeka 118, and Shanghai Tower. The world's tallest human-made structure is the Burj Khalifa in Dubai, with a height of 828 metres (2,717 ft). Learn about the debate over definition, the history of height records, and the categories of buildings and structures. Find out which building was the world's tallest in 2024, and see th

{'input': 'Find the heights of the world top 2 tallest building. Add the heights',
 'output': "The total height of the world's top 2 tallest buildings is 4790 feet."}

In [ ]:
agent_executor.invoke({"input": "What is the capital of Canada?"})

I should use the wikipedia tool to look up the capital of Canada.
Action: wikipedia
Action Input: "Capital of Canada"Page: Ottawa
Summary: Ottawa ( , ; Canadian French: [ɔtawɑ]) is the capital city of Canada. It is located in the southern portion  of the province of Ontario, at the confluence of the Ottawa River and the Rideau River. Ottawa borders Gatineau, Quebec, and forms the core of the Ottawa–Gatineau census metropolitan area (CMA) and the National Capital Region (NCR). As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest metropolitan area in Canada.
Ottawa is the political centre of Canada and the headquarters of the federal government. The city houses numerous foreign embassies, key buildings, organizations, and institutions of Canada's government; these include the Parliament of Canada, the Supreme Court, the residence of Canada's viceroy, and Office of the Prime Minister.
Founded

{'input': 'What is the capital of Canada?',
 'output': 'The capital of Canada is Ottawa.'}

In [ ]:
agent_executor.invoke({"input": "Whats the population there?"})

I should use the tools available to find the population of a specific location.
Action: DuckDuckGo Search
Action Input: "population of New York City"Basic Statistic New York-Newark-Jersey City metro area population in the U.S. 2010-2023 Premium Statistic Resident population of New York City, NY, by race 2022 New York, NY annual population growth rate. Between the years 2000 and 2023, i.e. 23 annual cycles, the population of New York grew year by year for 12 years and declined for 11 years. From 2000 to 2023, New York had an average annual population growth rate of 0.13% per year. Overall the population grew 3.03% between 2000 and 2023. Total population of the city in 2020 consists of 8,804,190 residents. The male population consists of 4,184,548 and females 4,619.642. "Between 2020 and 2021 the population of New York, NY grew from 8.38M to 8.74M, a 4.25% increase and its median household income grew from $67,046 to $70,663, a 5.39% increase" (DATAUSA) NYC Planning Population ... New Yo

{'input': 'Whats the population there?',
 'output': 'The population of New York City is 8,804,190 residents.'}

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
)

In [ ]:
template = '''Answer the following questions as best you can and improve your answers from previous answers in history. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
History: {chat_history}
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,memory=conversational_memory,verbose=True)

In [ ]:
agent_executor.invoke(
    {"input": "What is the capital of Canada?"},
)

I should search for the capital of Canada on Wikipedia to find the most accurate and up-to-date information.
Action: wikipedia
Action Input: "Capital of Canada"Page: Ottawa
Summary: Ottawa ( , ; Canadian French: [ɔtawɑ]) is the capital city of Canada. It is located in the southern portion  of the province of Ontario, at the confluence of the Ottawa River and the Rideau River. Ottawa borders Gatineau, Quebec, and forms the core of the Ottawa–Gatineau census metropolitan area (CMA) and the National Capital Region (NCR). As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest metropolitan area in Canada.
Ottawa is the political centre of Canada and the headquarters of the federal government. The city houses numerous foreign embassies, key buildings, organizations, and institutions of Canada's government; these include the Parliament of Canada, the Supreme Court, the residence of Canada's viceroy

{'input': 'What is the capital of Canada?',
 'chat_history': [],
 'output': 'The capital of Canada is Ottawa.'}

In [ ]:
agent_executor.invoke(
    {"input": "Whats the population there?"},
)

I should search for the population of Ottawa, Canada.
Action: DuckDuckGo Search
Action Input: 'Population of Ottawa, Canada'The city offers municipal services in both of Canada's official languages (Canadian English and Canadian French). [27] 367,035 people, or 36.45% of Ottawa's population, can speak both languages. [8] As such it is the largest city in Canada where municipal services are offered in both English and French. [28] As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest metropolitan area in Canada. [14][15] Ottawa is the political centre of Canada and the headquarters of the federal government. Table 17-10-0148-01 Population estimates, July 1, by census metropolitan area and census agglomeration, 2021 boundaries. Annual population estimates as of July 1st, by census metropolitan area and census agglomeration, single year of age, five-year age group and gender, based on the Stan

{'input': 'Whats the population there?',
 'chat_history': [HumanMessage(content='What is the capital of Canada?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of Canada is Ottawa.', additional_kwargs={}, response_metadata={})],
 'output': 'The population of Ottawa, Canada is 1,017,449 as of 2021.'}